In [ ]:
import os
os.system('sudo pip3 install cryptography==38.0.2 urllib3==1.26.6 s3fs boto3 pynubank')

### Bibliotecas

In [ ]:
import boto3
import json
from dateutil import parser
from datetime import timedelta, datetime, timezone
from pynubank import Nubank

In [ ]:
from cryptography.hazmat.backends.openssl.x509 import _Certificate

### SecretsManager

In [ ]:
#Criando um client com o secretsmanager
client = boto3.client('secretsmanager', region_name='us-east-1')

#Pegando as credenciais do rds
secret = client.get_secret_value(SecretId='nubank-autenticacao')

#Convertendo em um objeto JSON
chave = json.loads(secret['SecretString'])

### S3

In [ ]:
s3 = boto3.client('s3')

#Extraindo do S3 as credenciais das API
s3.download_file('nubank-api-dependencias', 'cert.p12', 'cert.p12')

### Nubank

##### Autenticacao

In [ ]:
#Definição de variáveis de autenticação do Nubank
cpf = chave['cpf']
senha = chave['senha']

In [ ]:
#Autenticação no Nubank usando o certificado
nu = Nubank()

In [ ]:
nu.authenticate_with_cert(cpf, senha, 'cert.p12')

###### Requets

In [ ]:
# Recupera as compras feitas no cartão
transacoes = nu.get_card_statements()

In [ ]:
# Recupera o saldo atual da conta
saldo = nu.get_account_balance()

In [ ]:
# Recupera o limite atual do credito
limite = nu.get_credit_card_balance()

In [ ]:
# Recupera o Feed de movimentacao da conta
feed = nu.get_account_feed_paginated()

##### Separando por assunto

In [ ]:
#Separa as transações em parcelamentos e compras
parcelamentos = []
compras = []

for transacao in transacoes:
    if 'details' in transacao and 'count' in transacao['details'].get('charges', {}):
        parcelamentos.append(transacao)
    else:
        compras.append(transacao)

In [ ]:
#Função para listar as parcelas de uma transação
def listar_parcelas(dados):
  lista = []
  for x in list(range(0,dados['details']['charges']['count'])):
    r = 1 + x
    datas = parser.parse(dados['time'])

    # Calcula a data de cada parcela adicionando 30 dias à data da transação
    new_data = datas + timedelta(days=30*r)
    new_data = new_data.strftime("%Y-%m-%dT%H:%M:%SZ")
    if x == 0:
      data = parser.parse(dados['time'])

      # Verifica se o dia da transação é menor que 9
      if data.day < 9:
        dic = {
                'time' : dados['time'],
                'title' : dados['title'],
                'description' : dados['description'],
                'category' : dados['category'],
                'amount' : dados['details']['charges']['amount']/100,
                'qtd_parcelas' : dados['details']['charges']['count'],
                'parcela' : x+1
            }
        lista.append(dic)
      else:
        dic_ = {
                'time' : new_data,
                'title' : dados['title'],
                'description' : dados['description'],
                'category' : dados['category'],
                'amount' : dados['details']['charges']['amount']/100,
                'qtd_parcelas' : dados['details']['charges']['count'],
                'parcela' : x+1
            }
        lista.append(dic_)
    else:
      dic_1 = {
          'time' : new_data,
          'title' : dados['title'],
          'description' : dados['description'],
          'category' : dados['category'],
          'amount' : dados['details']['charges']['amount']/100,
          'qtd_parcelas' : dados['details']['charges']['count'],
          'parcela' : x+1
          }
      lista.append(dic_1)
  return lista

In [ ]:
# Adicionando Data no dicionario do saldo
saldo = [{
    'Data':datetime.now().strftime('%Y:%m:%d %H:%M:%S'),
    'Saldo':saldo
}]

In [ ]:
# Montando o dic do tema Limite
limite = [{
    'Data':datetime.now().strftime('%Y:%m:%d %H:%M:%S'),
    'Fatura_atual':limite['open']/100,
    'Limite_disponivel':limite['available']/100,
    'limite':(limite['open']+limite['available'])/100
}]

In [ ]:
#Lista todas as parcelas de cada transação de parcelamento
parcelas = []
for h in parcelamentos:
  parcelas.extend(listar_parcelas(h))

In [ ]:
#Separando a lista feed em duas listas que representam receita e despesas
receita = []
despesas = []
r = ['Pagamento recebido','Transferência recebida','Resgate RDB']
for x in feed['edges']:
  if x['node']['title'] in r:
    receita.append(x['node'])
  else:
    despesas.append(x['node'])

#### Salvando or arquivos no S3

In [ ]:
#compras
s3.put_object(Body=json.dumps(compras), Bucket='nubank-api-dependencias', Key='JSONS/compras.json')

In [ ]:
#Parcelas
s3.put_object(Body=json.dumps(parcelas), Bucket='nubank-api-dependencias', Key='JSONS/parcelas.json')

In [ ]:
#saldo
s3.put_object(Body=json.dumps(saldo), Bucket='nubank-api-dependencias', Key='JSONS/saldo.json')

In [ ]:
#limite
s3.put_object(Body=json.dumps(limite), Bucket='nubank-api-dependencias', Key='JSONS/limite.json')

In [ ]:
#Despesas
s3.put_object(Body=json.dumps(despesas), Bucket='nubank-api-dependencias', Key='JSONS/despesas.json')

In [ ]:
#Resceitas
s3.put_object(Body=json.dumps(receita), Bucket='nubank-api-dependencias', Key='JSONS/receitas.json')